In [9]:
function inherit(p) {
  if (p == null) throw TypeError(); //p是一个对象，但不能是null
  if (Object.create)
    //如果Object.create()存在
    return Object.create(p); //直接使用它
  var t = typeof p; //否则进行进一步检测
  if (t !== "object" && t !== "function") throw TypeError();
  function f() {} //定义一个空构造函数
  f.prototype = p; //将其原型属性设置为p
  return new f(); //使用f()创建p的继承对象
}


In [10]:
//range.js:实现一个能表示值的范围的类
//这个工厂方法返回一个新的"范围对象"
function range(from, to) {
  //使用inherit()函数来创建对象,这个对象继承自在下面定义的原型对象
  //原型对象作为函数的一个属性存储,并定义所有"范围对象"所共享的方法(行为)
  var r = inherit(range.methods); //存储新的"范围对象"的起始位置和结束位置(状态)
  //这两个属性是不可继承的,每个对象都拥有唯一的属性
  r.from = from;
  r.to = to; //返回这个新创建的对象
  return r;
}
//原型对象定义方法,这些方法为每个范围对象所继承
range.methods = {
  //如果x在范围内,则返回true;否则返回false
  //这个方法可以比较数字范围,也可以比较字符串和日期范围
  includes: function(x) {
    return this.from <= x && x <= this.to;
  }, //对于范围内的每个整数都调用一次f
  //这个方法只可用做数字范围
  foreach: function(f) {
    for (var x = Math.ceil(this.from); x <= this.to; x++) f(x);
  }, //返回表示这个范围的字符串
  toString: function() {
    return "(" + this.from + "..." + this.to + ")";
  }
}; //这里是使用"范围对象"的一些例子
var r = range(1, 3); //创建一个范围对象
r.includes(2); //=>true:2在这个范围内
r.foreach(console.log); //输出1 2 3
console.log(r); //输出(1...3)




1
2
3
{ from: 1, to: 3 }


In [11]:
//range2.js:表示值的范围的类的另一种实现
//这是一个构造函数,用以初始化新创建的"范围对象"
//注意,这里并没有创建并返回一个对象,仅仅是初始化
function Range(from, to) {
  //存储"范围对象"的起始位置和结束位置(状态)
  //这两个属性是不可继承的,每个对象都拥有唯一的属性
  this.from = from;
  this.to = to;
}
//所有的"范围对象"都继承自这个对象
//注意,属性的名字必须是"prototype"
Range.prototype = {
  //如果x在范围内,则返回true;否则返回false
  //这个方法可以比较数字范围,也可以比较字符串和日期范围
  includes: function(x) {
    return this.from <= x && x <= this.to;
  }, //对于范围内的每个整数都调用一次f
  //这个方法只可用于数字范围
  foreach: function(f) {
    for (var x = Math.ceil(this.from); x <= this.to; x++) f(x);
  }, //返回表示这个范围的字符串
  toString: function() {
    return "(" + this.from + "..." + this.to + ")";
  }
}; //这里是使用"范围对象"的一些例子
var r = range(1, 3); //创建一个范围对象
r.includes(2); //=>true:2在这个范围内
r.foreach(console.log); //输出1 2 3
console.log(r); //输出(1...3)




1
2
3
{ from: 1, to: 3 }


In [12]:
var r = new Range(1,3)

In [13]:
r

{ from: 1, to: 3 }

In [14]:
r.foreach(console.log)

1
2
3


In [15]:
var F=function(){};//这是一个函数对象
var p=F.prototype;//这是F相关联的原型对象
var c=p.constructor;//这是与原型相关联的函数
c===F//=＞true:对于任意函数F.prototype.constructor==F

true

In [16]:
p

F {}

In [17]:
/*
*把p中的可枚举属性复制到o中,并返回o
*如果o和p中含有同名属性,则覆盖o中的属性
*这个函数并不处理getter和setter以及复制属性
*/
function extend(o, p) {
  for (prop in p) {
    //遍历p中的所有属性
    o[prop] = p[prop]; //将属性添加至o中
  }
  return o;
}



//一个用以定义简单类的函数
function defineClass(
  constructor, //用以设置实例的属性的函数
  methods, //实例的方法,复制至原型中
  statics //类属性,复制至构造函数中
) {
  if (methods) extend(constructor.prototype, methods);
  if (statics) extend(constructor, statics);
  return constructor;
}
//这是Range类的另一个实现
var SimpleRange = defineClass(
  function(f, t) {
    this.f = f;
    this.t = t;
  },
  {
    includes: function(x) {
      return this.f <= x && x <= this.t;
    },
    toString: function() {
      return this.f + "..." + this.t;
    }
  },
  {
    upto: function(t) {
      return new SimpleRange(0, t);
    }
  }
);




In [18]:
/*
*Complex.js:
*这个文件定义了Complex类,用来描述复数
*回忆一下,复数是实数和虚数的和,并且虚数i是-1的平方根
*/
/*
*这个构造函数为它所创建的每个实例定义了实例字段r和i
*这两个字段分别保存复数的实部和虚部
*它们是对象的状态
*/
function Complex(real, imaginary) {
  if (isNaN(real) || isNaN(imaginary))
    //确保两个实参都是数字
    throw new TypeError(); //如果不都是数字则抛出错误
  this.r = real; //复数的实部
  this.i = imaginary; //复数的虚部
} /*
*类的实例方法定义为原型对象的函数值属性
*这里定义的方法可以被所有实例继承,并为它们提供共享的行为
*需要注意的是,JavaScript的实例方法必须使用关键字this
*来存取实例的字段
*/
//当前复数对象加上另外一个复数,并返回一个新的计算和值后的复数对象
Complex.prototype.add = function(that) {
  return new Complex(this.r + that.r, this.i + that.i);
}; //当前复数乘以另外一个复数,并返回一个新的计算乘积之后的复数对象
Complex.prototype.mul = function(that) {
  return new Complex(
    this.r * that.r - this.i * that.i,
    this.r * that.i + this.i * that.r
  );
}; //计算复数的模,复数的模定义为原点(0,0)到复平面的距离
Complex.prototype.mag = function() {
  return Math.sqrt(this.r * this.r + this.i * this.i);
}; //复数的求负运算
Complex.prototype.neg = function() {
  return new Complex(-this.r, -this.i);
}; //将复数对象转换为一个字符串
Complex.prototype.toString = function() {
  return "{" + this.r + "," + this.i + "}";
}; //检测当前复数对象是否和另外一个复数值相等
Complex.prototype.equals = function(that) {
  return (
    that != null && //必须有定义且不能是null
    that.constructor === Complex && //并且必须是Complex的实例
    this.r === that.r &&
    this.i === that.i
  ); //并且必须包含相同的值
}; /*
*类字段(比如常量)和类方法直接定义为构造函数的属性
*需要注意的是,类的方法通常不使用关键字this,
*它们只对其参数进行操作
*/
//这里预定义了一些对复数运算有帮助的类字段
//它们的命名全都是大写,用以表明它们是常量
//(在ECMAScript 5中,还能设置这些类字段的属性为只读)
Complex.ZERO = new Complex(0, 0);
Complex.ONE = new Complex(1, 0);
Complex.I = new Complex(0, 1); //这个类方法将由实例对象的toString方法返回的字符串格式解析为一个Complex对象
//或者抛出一个类型错误异常
Complex.parse = function(s) {
  try {
    //假设解析成功
    var m = Complex._format.exec(s); //利用正则表达式进行匹配
    return new Complex(parseFloat(m[1]), parseFloat(m[2]));
  } catch (x) {
    //如果解析失败则抛出异常
    throw new TypeError("Can't parse'" + s + "'as a complex number.");
  }
}; //定义类的"私有"字段,这个字段在Complex.parse()中用到了
//下划线前缀表明它是类内部使用的,而不属于类的公有API的部分
Complex._format = /^\{([^,]+),([^}]+)\}$/;





/^\{([^,]+),([^}]+)\}$/

In [19]:
//用一个简单的函数创建简单的子类
function defineSubclass(
  superclass, //父类的构造函数
  constructor, //新的子类的构造函数
  methods, //实例方法:复制至原型中
  statics //类属性:复制至构造函数中
) {
  //建立子类的原型对象
  constructor.prototype = inherit(superclass.prototype);
  constructor.prototype.constructor = constructor; //像对常规类一样复制方法和类属性
  if (methods) extend(constructor.prototype, methods);
  if (statics) extend(constructor, statics); //返回这个类
  return constructor;
}
//也可以通过父类构造函数的方法来做到这一点
Function.prototype.extend = function(constructor, methods, statics) {
  return defineSubclass(this, constructor, methods, statics);
};






[Function]

In [20]:
function dog(age){
    var r = inherit(dog.methods);
    r.age = age
    return r
}

In [21]:
dog.methods = {
    'health': function(){
        if (this.age < 3){
            return true
        }
        else
            return false
    }
}

{ health: [Function: health] }

In [22]:
var d = dog(10)

In [23]:
d.health()

false

属性赋值操作首先检查原型链，以此判定是否允许赋值操作。例如，如果o继承自一个只读属性x，那么赋值操作是不允许的（6.2.3节将对此进行详细讨论）。

In [24]:
var unitcircle={r:1};//一个用来继承的对象
var c=inherit(unitcircle);//c继承属性r
c.x=1;c.y=1;//c定义两个属性
c.r=2;//c覆盖继承来的属性
unitcircle.r;//=＞1，原型对象没有修改

1

In [25]:
c.r

2

In [26]:
unitcircle.r

1

In [28]:
var o={//普通的数据属性
data_prop:10,//存取器属性都是成对定义的函数
get accessor_prop(){/*这里是函数体*/},
set accessor_prop(value){/*这里是函数体*/}
};

In [29]:
o

{ data_prop: 10, accessor_prop: [Getter/Setter] }

In [30]:
var o = {data_prop:10}

In [33]:
var o = {money: 10, get get_money(){return this.money + 10}, set set_money(value){ this.money = 11}}